# The goal of this notebook is:
To classify MNIST images using a reccurent neural network. To give a sequential flavor to a MNIST image
we treat each row of pixels as an element of a sequence. Thus, sine a MNIST image 28x28 pixels it will produce
a 28 long-sequence whose elements are comprised by 28 values.

In [1]:
import tensorflow as tf
from tensorflow.python.ops import rnn, rnn_cell
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

# Import MINST data
mnist = input_data.read_data_sets("/Users/optas/DATA/MNIST/", one_hot=True)

Extracting /Users/optas/DATA/MNIST/train-images-idx3-ubyte.gz
Extracting /Users/optas/DATA/MNIST/train-labels-idx1-ubyte.gz
Extracting /Users/optas/DATA/MNIST/t10k-images-idx3-ubyte.gz
Extracting /Users/optas/DATA/MNIST/t10k-labels-idx1-ubyte.gz


In [2]:
# Parameters
learning_rate = 0.001
training_iters = 100000
batch_size = 100

# Network Parameters
n_input = 28 # MNIST data input (img shape: 28*28)
n_steps = 28 # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [3]:
def RNN(x, weights, biases):
    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Permuting batch_size and n_steps
    x = tf.transpose(x, [1, 0, 2])
    # Reshaping to (n_steps*batch_size, n_input)
    x = tf.reshape(x, [-1, n_input])
    # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.split(0, n_steps, x)

    # Define an lstm cell with tensorflow
    lstm_cell = rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.initialize_all_variables()

In [4]:
display_step = 10   # Every display_step * batch_size examples show results.
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc)
        step += 1
    print "Optimization Finished!"

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print "Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label})

Iter 1000, Minibatch Loss= 1.717847, Training Accuracy= 0.48000
Iter 2000, Minibatch Loss= 1.324344, Training Accuracy= 0.53000
Iter 3000, Minibatch Loss= 1.160099, Training Accuracy= 0.57000
Iter 4000, Minibatch Loss= 0.828457, Training Accuracy= 0.77000
Iter 5000, Minibatch Loss= 0.771893, Training Accuracy= 0.75000
Iter 6000, Minibatch Loss= 0.812535, Training Accuracy= 0.71000
Iter 7000, Minibatch Loss= 0.504349, Training Accuracy= 0.83000
Iter 8000, Minibatch Loss= 0.506688, Training Accuracy= 0.86000
Iter 9000, Minibatch Loss= 0.867400, Training Accuracy= 0.78000
Iter 10000, Minibatch Loss= 0.470656, Training Accuracy= 0.83000
Iter 11000, Minibatch Loss= 0.513396, Training Accuracy= 0.83000
Iter 12000, Minibatch Loss= 0.385232, Training Accuracy= 0.90000
Iter 13000, Minibatch Loss= 0.342705, Training Accuracy= 0.88000
Iter 14000, Minibatch Loss= 0.254422, Training Accuracy= 0.93000
Iter 15000, Minibatch Loss= 0.301832, Training Accuracy= 0.92000
Iter 16000, Minibatch Loss= 0.5654